In [1]:
import numpy as np
import pandas as pd
import random
import tensorflow as tf
import matplotlib.pyplot as plt
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from sklearn.metrics import confusion_matrix, accuracy_score
from tensorflow.keras.losses import categorical_crossentropy

In [2]:
data = pd.read_csv('iris.data')
print("Rows:",len(data))
print("Classes:",set(data['iris']))
samples = data.to_numpy()[:,:4] 
labels = data.to_numpy()[:,4]
samples = samples.astype(float)
from sklearn.preprocessing import LabelBinarizer
lb = LabelBinarizer()
labels = lb.fit_transform(labels)
print(labels.shape)
import sklearn.model_selection
(trainSamples, testSamples, trainLabels, testLabels) = sklearn.model_selection.train_test_split(samples, labels)
testSamples.shape

Rows: 150
Classes: {'Iris-setosa', 'Iris-versicolor', 'Iris-virginica'}
(150, 3)


(38, 4)

In [3]:
def build_model():
    model = Sequential()
    model.add(Dense(4, input_dim=4, activation='sigmoid'))
    model.add(Dense(3, activation='softmax'))
    for layer in model.layers:
        layer.built = True
    return model
model = build_model()
model.summary()


Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 4)                 20        
_________________________________________________________________
dense_1 (Dense)              (None, 3)                 15        
Total params: 35
Trainable params: 35
Non-trainable params: 0
_________________________________________________________________


In [4]:
def build_hyper_model(): 
    hyper_model = tf.keras.models.Sequential(name='hyper_model')
    hyper_model.add(Dense(16,input_dim=4)) ##,activation='relu') )
    hyper_model.add(Dense(35,activation='tanh'))
    return hyper_model
hyper_model = build_hyper_model()
hyper_model.summary()

Model: "hyper_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense_2 (Dense)              (None, 16)                80        
_________________________________________________________________
dense_3 (Dense)              (None, 35)                595       
Total params: 675
Trainable params: 675
Non-trainable params: 0
_________________________________________________________________


In [5]:
for i,layer in enumerate(model.layers):
    print(i,layer.kernel.shape)
    print(i,layer.bias.shape)

0 (4, 4)
0 (4,)
1 (4, 3)
1 (3,)


In [6]:
def apply_weights(model, weights):
    #layer 0
    w0 = weights[0:16]
    w0 = tf.reshape(w0, [4,4]) 
    model.layers[0].kernel = w0
    b0 = weights[16:20]
    model.layers[0].bias = b0
    #layer 1
    w1 = weights[20:32]
    w1 = tf.reshape(w1, [4,3]) 
    model.layers[1].kernel = w1
    b1 = weights[32:]
    model.layers[1].bias = b1


In [7]:
def hyper_step(sample, label):
    with tf.GradientTape() as tape:
        predicted_weights = hyper_model(sample)
        predicted_weights = tf.reshape(predicted_weights,[-1])
        apply_weights(model,predicted_weights)
        predictions = model(sample)
        loss = categorical_crossentropy(label,predictions)
    gradients = tape.gradient(loss, hyper_model.trainable_variables)
    opt.apply_gradients(zip(gradients, hyper_model.trainable_variables))
    return loss

In [8]:
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score, cohen_kappa_score
opt = tf.keras.optimizers.Adam(0.001)
print("Start")
hyper_model = build_hyper_model()
model = build_model()
EPOCHS = 100
step_no=0
for e in range(EPOCHS):
    step_no+=1
    num_samples_per_epoch = 200
    print(".",end='')
    losses = []
    for i in range(num_samples_per_epoch):
        idx = np.random.randint(low=0, high=trainSamples.shape[0], size=1)
        tsample = trainSamples[idx]
        tlabel = trainLabels[idx]
        ##print(tsample.shape)=
        loss=hyper_step(tsample,tlabel)
        losses.append(loss)
        
    if e%10==0:
        losses = np.array(losses)
        print("loss",losses.mean())
        trainResults = model.predict(trainSamples)
        trainAccuracy = accuracy_score(trainLabels.argmax(axis=1), trainResults.argmax(axis=1))
        print(f'Train accuracy: {trainAccuracy:.2f}')

print('Done')





Start
.loss 0.5681379
Train accuracy: 0.37
..........loss 0.15388747
Train accuracy: 0.37
..........loss 0.10879514
Train accuracy: 0.37
..........loss 0.10255317
Train accuracy: 0.37
..........loss 0.10840687
Train accuracy: 0.37
..........loss 0.064498335
Train accuracy: 0.37
..........loss 0.06684572
Train accuracy: 0.37
..........loss 0.06931856
Train accuracy: 0.37
..........loss 0.09645128
Train accuracy: 0.37
..........loss 0.05617383
Train accuracy: 0.37
.........Done
